<a href="https://colab.research.google.com/github/Meenakshigupta6527/Object-Character-Recognition-OCR-/blob/main/Custom_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Setup Environment
#### First, connect Google Drive to Colab to store the dataset, model weights, and results.

In [4]:
## Mounting Google Drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Installing Required Dependencies

In [5]:
!sudo apt install tesseract-ocr
!pip install pytesseract opencv-python numpy matplotlib ultralytics


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (9,443 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

## 2. Data Preparation

In [6]:
dataset_path = "/content/drive/MyDrive/Custom_OCR/Dataset_OCR"

## 3. Model Training
### Task 3.1: Train YOLOv3 Model



In [7]:
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
!make


Cloning into 'darknet'...
remote: Enumerating objects: 15873, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 15873 (delta 3), reused 0 (delta 0), pack-reused 15863 (from 3)
Receiving objects: 100% (15873/15873), 14.47 MiB | 13.47 MiB/s, done.
Resolving deltas: 100% (10686/10686), done.
/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ []8;;https://gcc.gnu.org/onlinedocs/gcc/W

#### 3.2 Move Pretrained Weights

In [8]:
!wget https://pjreddie.com/media/files/yolov3.weights -O yolov3.weights


--2025-03-20 13:32:38--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  22.1MB/s    in 8.5s    

2025-03-20 13:32:47 (27.8 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



#### 3.3 Train YOLOv3 on Custom Data

In [16]:
!./darknet detector train /content/drive/MyDrive/Custom_OCR/yolo_config/obj.data \
/content/drive/MyDrive/Custom_OCR/yolo_config/yolov3.cfg \
/content/drive/MyDrive/Custom_OCR/yolo_config/yolov3.weights

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
yolov3
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
  10 conv 

## 4. Inference and Post-Processing
#### 4.1 Load and Detect Objects in an Image

In [28]:
import cv2
import numpy as np

def detect_objects(image_path, output_image_path):
    # Load YOLOv3 model
    net = cv2.dnn.readNet("/content/drive/MyDrive/Custom_OCR/yolo_config/yolov3.weights",
                          "/content/drive/MyDrive/Custom_OCR/yolo_config/yolov3.cfg")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    # Load image
    image = cv2.imread(image_path)
    height, width, channels = image.shape

    # Prepare image for YOLO
    blob = cv2.dnn.blobFromImage(image, scalefactor=1/255, size=(416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layers)

    # Process detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # Confidence threshold
                center_x, center_y, w, h = (obj[0:4] * np.array([width, height, width, height])).astype("int")
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Draw bounding box
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Save detected image
    cv2.imwrite(output_image_path, image)


##  5. OCR Using Tesseract
#### 5.1 Preprocess Image for OCR

In [24]:
import pytesseract

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return thresholded

processed_image = preprocess_image("/content/drive/MyDrive/Custom_OCR/detected.jpg")
cv2.imwrite("/content/drive/MyDrive/Custom_OCR/preprocessed.jpg", processed_image)


True

In [25]:
text = pytesseract.image_to_string(processed_image)
print("Extracted Text:\n", text)

# Save to CSV
import pandas as pd
df = pd.DataFrame({"Extracted Text": [text]})
df.to_csv("/content/drive/MyDrive/Custom_OCR/output.csv", index=False)


Extracted Text:
 (Canparate Cfien : Terecare Vochnatagies United © 0 37/3, TTC MDC. Teste, ta
© O22 3090. 0000 / 4125 2525 © 9691966066 a weten@unrerese.som

es ¢ ASHIDM SENGUPTA (S8¥/6)
amo 1 ser

Vast agmee =| anocvaM C

 



In [26]:
ground_truth = "Expected text from lab report..."
accuracy = sum(1 for a, b in zip(text, ground_truth) if a == b) / len(ground_truth)
print(f"OCR Accuracy: {accuracy * 100:.2f}%")


OCR Accuracy: 3.12%


In [32]:
import os

import pytesseract

def extract_text(image_path):
    """Extract text from the preprocessed image using Tesseract."""
    text = pytesseract.image_to_string(image_path, config="--psm 6")
    return text


def process_lab_report(image_path):
    detected_img = "/content/drive/MyDrive/Custom_OCR/detected_output.jpg"

    # Step 1: Detect objects
    detect_objects(image_path, detected_img)

    # Step 2: Preprocess Image
    preprocessed_img = preprocess_image(detected_img)

    # Step 3: Extract text
    extracted_text = extract_text(preprocessed_img)

    return extracted_text

image_path = "/content/drive/MyDrive/Custom_OCR/Dataset_OCR/thyrocare_0_2649.jpg"
result_text = process_lab_report(image_path)
print("Final Extracted Text:", result_text)



Final Extracted Text: | a: Thyn
r 18, AAA/36, Reghunsthour,
KoRate-700 059 Tee:
Conparate Offer : Tierocare Yochastagies Uniaed 9 0 37/3, TTC MADC, Teste, tani
+ @ G22 30900000 / 4125 2525 © 9691066006 wel @wnreceesom -
ry Leh
—« 1 ASHI SENGUPTA (Sav/M) saswun COULECTED AT 3
amo 1 Seu (7824357319). KALPANA MEDICOS AMD
Vast agape | aanocvanc UAB,BINUTOLT ROAD ,HOUAL ABBAM :
'
TaTtwe TECHNOLOGY van ars: =.
‘AUKALINE PHOSPHATASE PHOTOMETRY 67 wt =
G@mRUBTN - TOTAL PHOTOMETRY 0.87 wwe oven:
SRLIRUBIN -DIRECT PHOTOMETRY 0.18 mea <0h
GRIRUER (INDIRECT) CALCULATED. O39 mee fe
D @NOTAA GLUTAMYL TRANSFERASE (GG!) PHOTOMETRY M3 mw «
" ABPARTATE AMINOTRANSFERASE {SGOT ) PHOTOMETRY 1 uw
MABEL TRANSAMINASE (SGPT) PHOTOMETRY m9 mw -
PROTEMS - TOTAL PHOTOMETRY r4 owe .
= as
ARE - SERN PHOTOMETRY 3.93 owe oy
SERUM MB/GLOOUALIN RATIO CALCLIAATED: 1.22 nate =
‘a ae mroromenty _3.1 pie
BA pinnse copratate with clinical conditions.
al |



In [34]:
result_text = process_lab_report("/content/drive/MyDrive/Custom_OCR/Dataset_OCR/thyrocare_0_1861.jpg")
print(result_text)


4
- Thyn
; TL, RAA/36, Raghunathpur, =
, Kolkate-700 059 ‘Tein Thea
(Comparate Office : Tuyrocare Technologies United © D-37/3, TTC MIOC. Terbhn, Mand
© 022 - 10900000 / 4125 2525 © 0691066006 @ wetnen@unwocirecom a
a
‘ Ll
mon 5 ASHIM SENGUPTA (S8V/M) Sanur COULECTED AT 1 "|
an ov 1 SEF (7024357519), KALPANA meDeCOS ap coy
FAST AGKED: AAROGYANC LAS,GIMITOLI ROAD .HOUAS ASEAN
17
‘TEST MADE TECHNOLOGY VALUE wars a
AMALRALINE PHOSPHATASE PHOTOMETRY 67 we > ae
GRIRUBIN - TOTAL PHOTOMETRY 0.57 mod arse,

, SRIRAM -DIRECT PHOTOMETRY 0.18 me y
SRIWUBI (INDIRECT) CALCULATED 0.39 me on
GIPEU GUTAMML. TRANSFERASE (GG1) PHOTOMETRY “a w «< a
AGPARTATE AMINOTRANSFERASE (SGOT ) PHOTOMETRY 33.1 wi <
MANINE TRANSAMINASE (SGPT) PHOTOMETRY 38.9 wi oT

- DROTERE - TOTAL PHOTOMETRY nv ove =
ALBUIEN - SERUM PHOTOMETRY 3.93 owe <u
SERUM ALB/GLOBULIN RATIO CALCULATED 1.22 fatto ee.

'  $RRUN GLOBULIN ___ PHOTOMETRY 3.28 m

Bien eeeaiehe auittn rinicel raonditions.

